In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 1. Import the dataset and clean column names
data = pd.read_csv("RealEstate_Prices.csv")
print("First few rows of the dataset:")
print(data.head())

In [ ]:
# Clean column names by removing spaces, special characters, and renaming for clarity
data.columns = data.columns.str.strip().str.replace(' ', '_').str.replace(r'[^\w\s]', '')
print("\nCleaned Column Names:")
print(data.head())

In [ ]:
# 2. Handle missing values
# Decide on a strategy to handle missing values. Example: filling numerical with mean and categorical with mode.
for column in data.columns:
    if data[column].dtype == 'object':  # Check if column is categorical
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:  # Numeric column
        data[column].fillna(data[column].mean(), inplace=True)

# Display summary of missing values to ensure they are handled
print("\nMissing Values after Imputation:")
print(data.isnull().sum())

In [ ]:
# 4. Filter and subset data based on criteria
# Example: Filter properties sold after 2015 and of type 'Single Family'
filtered_data = data[(data['Year_Built'] > 2015) & (data['Property_Type'] == 'Single Family')]
print("\nFiltered Data:")
print(filtered_data.head())


In [ ]:
# 5. Handle categorical variables by encoding
# Example of one-hot encoding categorical variables
encoded_data = pd.get_dummies(filtered_data, columns=['Property_Type', 'Location'], drop_first=True)
print("\nCurrent Columns in encoded_data:")
print(encoded_data.columns)

In [ ]:
# 6. Aggregate data to calculate summary statistics
# Calculate average sale price by neighborhood
avg_price_by_neighborhood = data.groupby('Neighborhood').agg({'Sale_Price': 'mean'}).reset_index()
avg_price_by_neighborhood.rename(columns={'Sale_Price': 'Average_Sale_Price'}, inplace=True)

In [ ]:
print("\nAverage Sale Price by Neighborhood:")
print(avg_price_by_neighborhood)

In [ ]:
# 7. Identify and handle outliers
# Using IQR to identify outliers in 'Sale_Price'
Q1 = encoded_data['Sale_Price'].quantile(0.25)
Q3 = encoded_data['Sale_Price'].quantile(0.75)
IQR = Q3 - Q1


In [ ]:
# Define limits for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


In [ ]:
# Filter out outliers
outlier_filtered_data = encoded_data[(encoded_data['Sale_Price'] >= lower_bound) & (encoded_data['Sale_Price'] <= upper_bound)]

In [ ]:
# Display number of rows before and after outlier removal
print("\nNumber of rows before outlier removal:", len(encoded_data))
print("Number of rows after outlier removal:", len(outlier_filtered_data))

In [ ]:
# Save cleaned data to a new file
outlier_filtered_data.to_csv("Cleaned_RealEstate_Data.csv", index=False)

print("\nData wrangling completed! Cleaned dataset saved as 'Cleaned_RealEstate_Data.csv'.")